## Step 1: Training the Models

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt
!ls -lh {HOME}/weights

In [2]:
pip install ultralytics

  Using cached ultralytics-8.2.97-py3-none-any.whl.metadata (39 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.6-py3-none-any.whl.metadata (9.1 kB)
Using cached ultralytics-8.2.97-py3-none-any.whl (873 kB)
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----------- ---------------------------- 4.7/15.8 MB 23.8 MB/s eta 0:00:01
   ---------------------------------------  15.7/15.8 MB 41.2 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 38.3 MB/s eta 0:00:00
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Using cached ultralytics_thop-2.0.6-py3-none

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 0.17.6 requires numpy==1.21.5, but you have numpy 1.26.4 which is incompatible.


In [3]:
pip show ultralytics

Name: ultralytics
Version: 8.2.97
Summary: Ultralytics YOLO for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: d:\anaconda\envs\d2l\lib\site-packages
Requires: matplotlib, numpy, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, torch, torchvision, tqdm, ultralytics-thop
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## 1.1 图片数据训练

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

# 设置权重文件夹路径
weights_dir = os.path.join(os.path.expanduser('~'), 'Desktop', 'weights')

# 初始化 YOLO 模型
yolov8_models = {
    "yolov8n": YOLO(os.path.join(weights_dir, 'yolov8n.pt')),
    "yolov8s": YOLO(os.path.join(weights_dir, 'yolov8s.pt')),
    "yolov8x": YOLO(os.path.join(weights_dir, 'yolov8x.pt'))
}

# Path to the dataset configuration file (YAML)
DATASET_CONFIG = r'C:\Users\ASUS\Desktop\trafic_data\data_1.yaml'

# Directory to save the training graphs for all models
main_folder = 'training-graphs'
if not os.path.exists(main_folder):  # 自动帮我们创立好文件夹
    os.makedirs(main_folder)  # 每个模型的文件里面总共两个图（一个accuracy，一个loss）

# Train each of the four models
for model_name, model in yolov8_models.items():
    print(f"Training {model_name}...")

    # Directory for saving this model's training graphs
    model_folder = os.path.join(main_folder, model_name)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    
    # Train the model
    results = model.train(
        data=DATASET_CONFIG,   # path to the YAML config file
        epochs=10,             # number of training epochs
        batch=128,              # batch size
        imgsz=640,             # image size for training (640x640)
        name=model_name,       # name for this experiment run
        workers=8,             # number of data loading workers （or processes 进程 per time)
        plots=True,            # save loss and mAP plots
        device='0'             # 0: GPU, 1: CPU
    )

    # Retrieve training metrics (loss, mAP, etc.)
    metrics = results.metrics

    # Define the range of epochs
    epochs = range(1, 11)  # Adjust according to your number of epochs

    # Loss plot
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics['train/loss'], label='Training Loss')
    plt.plot(epochs, metrics['val/loss'], label='Validation Loss')
    plt.title(f'{model_name} - Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    loss_file_path = os.path.join(model_folder, f'{model_name}_loss.png')
    plt.savefig(loss_file_path)
    plt.close()

    # Accuracy (mAP) plot
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics['train/map50'], label='Training mAP@0.5')
    plt.plot(epochs, metrics['val/map50'], label='Validation mAP@0.5')
    plt.title(f'{model_name} - Accuracy (mAP) Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('mAP@0.5')
    plt.legend()
    plt.grid(True)
    accuracy_file_path = os.path.join(model_folder, f'{model_name}_accuracy.png')
    plt.savefig(accuracy_file_path)
    plt.close()

    print(f"Training completed for {model_name} and plots saved in {model_folder}.")

print("All models have been trained and plots saved.")

## 1.2 视频类型训练数据